In [5]:
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.optimization import SingleMetricOptimizer, Gingleator
from gerrychain.tree import recursive_seed_part
from functools import partial
import pandas as pd
import json
from networkx.readwrite import json_graph
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import random

In [6]:
# Set random seed
random.seed(2024)

# Load the graph
graph = Graph.from_json("./VA/VA1.json")

In [10]:
# Population and district settings
POPCOL = "TOTPOP"
SEN_DISTS = 40
EPS = 0.1  
TOTPOP = sum(graph.nodes()[n][POPCOL] for n in graph.nodes())

# Create updaters 
chain_updaters = {
    "population": updaters.Tally(POPCOL, alias="population"),
    "VAP": updaters.Tally("VAP"),
    "BVAP": updaters.Tally("BVAP"),   # Needed for minority calculation
}


In [11]:
# Initial random partition
initial_partition = Partition.from_random_assignment(
    graph=graph,
    n_parts=SEN_DISTS,
    epsilon=EPS,
    pop_col=POPCOL,
    updaters=chain_updaters
)

RuntimeError: Could not find a possible cut after 10000 attempts.

In [ ]:
# ReCom proposal with pair reselection to avoid BipartitionWarning
proposal = partial(
    recom,
    pop_col=POPCOL,
    pop_target=TOTPOP/SEN_DISTS,
    epsilon=EPS,
    node_repeats=1,
    allow_pair_reselection=True
)

In [ ]:
# Constraints: population constraint only (optional: add compactness if you want)
population_constraint = constraints.within_percent_of_ideal_population(initial_partition, EPS)


In [ ]:
from gingleator import Gingleator

In [ ]:
# Instantiate Gingleator
gingleator = Gingleator(
    initial_partition=initial_partition,
    threshold=0.4,                     # 40% BVAP as opportunity district threshold
    score_funct=Gingleator.reward_partial_dist,  # Reward based on partial success
    minority_perc_col="BVAP",
    pop_col="TOTPOP",
    epsilon=EPS
)

In [ ]:
# Important: initialize minority % updater
gingleator.init_minority_perc_col(
    minority_pop_col="BVAP",
    total_pop_col="TOTPOP",
    minority_perc_col="BVAP"
)

In [ ]:
# Run Gingleator Short Bursts
num_bursts = 1000
num_steps_per_burst = 10

# Perform short burst run
(best_partition, observed_scores) = gingleator.short_burst_run(
    num_bursts=num_bursts,
    num_steps=num_steps_per_burst,
    verbose=True,     # Show progress
    maximize=True
)

In [ ]:
# Plotting
flattened_scores = observed_scores.flatten()

fig, ax = plt.subplots(figsize=(12, 6))
plt.plot(flattened_scores, label="Gingleator Short Burst (reward_partial_dist)")
plt.xlabel("Steps", fontsize=16)
plt.ylabel("Gingles Score", fontsize=16)
plt.legend()
plt.title("Short Burst Gingleator Optimization", fontsize=18)
plt.show()